In [1]:
import os

project_path = r"C:\Users\user\anaconda3\envs\Chicken-disease-Classification-Project"

os.chdir(project_path)
print(f"Moved to: {os.getcwd()}")


Moved to: C:\Users\user\anaconda3\envs\Chicken-disease-Classification-Project


In [2]:
%pwd

'C:\\Users\\user\\anaconda3\\envs\\Chicken-disease-Classification-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path                          # Not in YAML – likely set in code
    base_model_dir: Path                    # Not in YAML – likely set in code
    updated_base_model_dir: Path            # Not in YAML – likely set in code
    params_image_size: List[int]            # From YAML → MODEL_PARAMS.IMAGE_SIZE
    params_learning_rate: float             # From YAML → MODEL_PARAMS.LEARNING_RATE
    params_include_top: bool                # From YAML → MODEL_PARAMS.INCLUDE_TOP
    params_weights: str                     # From YAML → MODEL_PARAMS.WEIGHTS
    params_classes: int 

In [4]:
from cnnclassifier.constants import *
from cnnclassifier.utils.common import  read_yaml, create_directories # Assumed utility file

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAM_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config['artifacts_root']])  # ✅ This is correct

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config['prepare_base_model']

        create_directories([config['root_dir']])  # ✅ Treat as dictionary

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config['root_dir']),
            base_model_dir=Path(config['base_model_dir']),
            updated_base_model_dir=Path(config['updated_base_model_dir']),
            params_image_size=self.params['MODEL_PARAMS']['IMAGE_SIZE'],
            params_learning_rate=self.params['MODEL_PARAMS']['LEARNING_RATE'],
            params_include_top=self.params['MODEL_PARAMS']['INCLUDE_TOP'],
            params_weights=self.params['MODEL_PARAMS']['WEIGHTS'],
            params_classes=self.params['MODEL_PARAMS']['CLASSES']
        )

        return prepare_base_model_config

In [6]:
# component
import os
import urllib.request as request
from zipfile import ZipFile, BadZipFile
import tensorflow as tf

In [7]:
import tensorflow as tf
from pathlib import Path

class PrepareBaseModel:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.full_model = None

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        # Freeze all layers
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        # Freeze up to a specific layer
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:freeze_till]:
                layer.trainable = False

        # Add custom head
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(classes, activation='softmax')(flatten_in)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)

        # Compile
        full_model.compile(
            loss='categorical_crossentropy',
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            metrics=['accuracy']
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = PrepareBaseModel._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        PrepareBaseModel.save_model(
            path=self.config.updated_base_model_dir,
            model=self.full_model
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


In [ ]:
#pipeline
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e